In [171]:
# Identify athletes meeting World Athletics selection rules: 

#Qualifying window: 

#    For Marathon and 35km Race Walks: 5 Nov 2023 to 4 May 2025 

#    For 10,000m, 20km RW, Relays & Combined Events: 25 Feb 2024 to 24 Aug 2025 

#    For all other events: 1 Aug 2024 to 24 Aug 2025 


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [172]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic
import dateutil.parser as parser 
import datacompy



from google.cloud import storage



In [759]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [760]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,18-Feb-98,SGP,2.5,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,18-Feb-98,SGP,-1.9,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,2002,SGP,None,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,2003,SGP,None,Sprint,Male,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,25-Feb-92,SGP,None,Throw,Male,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31094,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31095,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31096,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31097,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International


In [761]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/World Athletics Champs/')


competitors.to_csv('athletes_downloaded_database.csv', sep=',', encoding='utf-8-sig', index=False)

In [762]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,18-Feb-98,SGP,2.5,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,18-Feb-98,SGP,-1.9,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,2002,SGP,None,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,2003,SGP,None,Sprint,Male,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,25-Feb-92,SGP,None,Throw,Male,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31094,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31095,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31096,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31097,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International


In [763]:
competitors[competitors['EVENT']=='Marathon']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
5,"He, Yong",02:33:16,None,None,294,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
9,"HE, HENRY YONG",2:40:46,Individual,40,None,None,Marathon,None,None,None,1983,None,-,Marathon,Male,Hangzhou Marathon,17-Dec,2023,International
10,"WONG, MELVIN",2:36:27,Individual,40,None,None,Marathon,None,None,None,1983,None,-,Marathon,Male,Valencia Marathon,3-Dec,2023,International
11,"TAN, AARON JUSTIN WEN JIE",2:36:22,Individual,30,None,None,Marathon,None,None,None,1993,None,-,Marathon,Male,Valencia Marathon,3-Dec,2023,International
12,SIEW LU CHNG,2:59:45,Individual,43,None,None,Marathon,None,None,None,1980,None,-,Marathon,Female,Valencia Marathon,3-Dec,2023,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31094,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31095,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31096,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International
31097,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International


In [764]:
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.loc[rowIndex,'DATE']
    year = competitors.loc[rowIndex,'YEAR']    
    
    if 'to' in date or ' - ' in date:
        
        if re.search('to|\s\-\s\d\s|\s\-\d\d', date):  # e.g. 03-04
              
            pos = re.search('to|\s\-\s\d', date)
            # Splice string to day and month

            split_pos_start=pos.start()+3
            


            final_date = date[split_pos_start:] # left string post splicing

            print(i, pos, date, final_date)
            final_year = year[2:]

            event_date = final_date + '/' + final_year

            print('old code', date, final_date, event_date)

            competitors.loc[rowIndex, 'event_date'] = event_date

        elif re.search('(\-\s\d\w)|(\-\s\d\d\w)', date):  # e.g. 18 - 19 January
                        
            pos = re.search('\-', date)  # from '-' onwards only
            # Splice string to day and month

            split_pos_start=pos.start()+2
            

            final_date = date[split_pos_start:] # left string post splicing

            
            final_year = year[2:]

            event_date = final_date + ' ' + final_year


            competitors.loc[rowIndex, 'event_date'] = event_date
            
        
    elif re.search('\w\-\w', date):
        
        if competitors.loc[rowIndex, 'COMPETITION'] == "National School Games":
            
            if competitors.loc[rowIndex, 'YEAR'] == '2024':
        
                event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd. 04 because event was in April 24 only
            
       #         print('NSG 2024', event_date)
        
                competitors.loc[rowIndex, 'event_date'] = event_date
            
            elif competitors.loc[rowIndex, 'YEAR'] == '2025':
                
                event_date = date + '-' +year[2:]
                
        #        print('NSG2025', event_date)
                
                competitors.loc[rowIndex, 'event_date'] = event_date
                
        elif re.search('\d\-\d',  date):        #10-13 April
            
            print('HERE', i, date)

            rpos = re.search('\-', date)
            string = date[rpos.end():]
            
            print('extracted date', string)
            
            event_date = string + ' ' + year
            
            print('event date', event_date)
            
            competitors.loc[rowIndex, 'event_date'] = event_date

            
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        
        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       


43 <re.Match object; span=(1, 5), match=' - 4'> 1 - 4 May 4 May
old code 1 - 4 May 4 May 4 May/25
44 <re.Match object; span=(1, 5), match=' - 4'> 1 - 4 May 4 May
old code 1 - 4 May 4 May 4 May/25
45 <re.Match object; span=(1, 5), match=' - 4'> 1 - 4 May 4 May
old code 1 - 4 May 4 May 4 May/25
46 <re.Match object; span=(1, 5), match=' - 4'> 1 - 4 May 4 May
old code 1 - 4 May 4 May 4 May/25
47 <re.Match object; span=(1, 5), match=' - 4'> 1 - 4 May 4 May
old code 1 - 4 May 4 May 4 May/25
48 <re.Match object; span=(1, 5), match=' - 4'> 1 - 4 May 4 May
old code 1 - 4 May 4 May 4 May/25
49 <re.Match object; span=(1, 5), match=' - 4'> 1 - 4 May 4 May
old code 1 - 4 May 4 May 4 May/25
HERE 102 10-13 April
extracted date 13 April
event date 13 April 2025
HERE 103 10-13 April
extracted date 13 April
event date 13 April 2025
109 <re.Match object; span=(2, 6), match=' - 1'> 12 - 13 October 13 October
old code 12 - 13 October 13 October 13 October/24
110 <re.Match object; span=(2, 6), match=' - 1'>

7407 <re.Match object; span=(1, 5), match=' - 5'> 3 - 5 May 5 May
old code 3 - 5 May 5 May 5 May/24
7460 <re.Match object; span=(1, 5), match=' - 3'> 1 - 3 March 3 March
old code 1 - 3 March 3 March 3 March/24
7461 <re.Match object; span=(1, 5), match=' - 3'> 1 - 3 March 3 March
old code 1 - 3 March 3 March 3 March/24
7485 <re.Match object; span=(2, 6), match=' - 1'> 12 - 13 October 13 October
old code 12 - 13 October 13 October 13 October/24
7486 <re.Match object; span=(2, 6), match=' - 1'> 12 - 13 October 13 October
old code 12 - 13 October 13 October 13 October/24
7487 <re.Match object; span=(2, 6), match=' - 1'> 12 - 13 October 13 October
old code 12 - 13 October 13 October 13 October/24
7488 <re.Match object; span=(2, 6), match=' - 1'> 12 - 13 October 13 October
old code 12 - 13 October 13 October 13 October/24
7489 <re.Match object; span=(2, 6), match=' - 1'> 12 - 13 October 13 October
old code 12 - 13 October 13 October 13 October/24
7490 <re.Match object; span=(2, 6), match=' -

9094 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9095 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9096 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9097 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9098 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9099 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9100 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9101 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9102 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
9103 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/2

10644 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10645 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10646 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10647 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10648 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10649 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10650 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10651 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10652 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10653 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
10654 <re.Match object; span=(

12341 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12342 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12343 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12344 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12345 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12346 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12347 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12348 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12349 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
12350 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25


...

22538 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22539 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22540 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22541 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22542 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22543 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22544 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22545 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22546 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22547 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
22548 <re.Match object; span=(4, 6), match='to'> 3

24220 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24221 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24222 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24223 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24224 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24225 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24226 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24227 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24228 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25
24229 <re.Match object; span=(4, 6), match='to'> 1/8 to 1/12 1/12
old code 1/8 to 1/12 1/12 1/12/25


25922 <re.Match object; span=(5, 7), match='to'> 11/9 to 11/10 11/10
old code 11/9 to 11/10 11/10 11/10/24
25923 <re.Match object; span=(5, 7), match='to'> 11/9 to 11/10 11/10
old code 11/9 to 11/10 11/10 11/10/24
25924 <re.Match object; span=(5, 7), match='to'> 11/9 to 11/10 11/10
old code 11/9 to 11/10 11/10 11/10/24
25925 <re.Match object; span=(5, 7), match='to'> 11/9 to 11/10 11/10
old code 11/9 to 11/10 11/10 11/10/24
25926 <re.Match object; span=(5, 7), match='to'> 11/9 to 11/10 11/10
old code 11/9 to 11/10 11/10 11/10/24
25927 <re.Match object; span=(5, 7), match='to'> 11/9 to 11/10 11/10
old code 11/9 to 11/10 11/10 11/10/24
25928 <re.Match object; span=(5, 7), match='to'> 11/9 to 11/10 11/10
old code 11/9 to 11/10 11/10 11/10/24
25929 <re.Match object; span=(5, 7), match='to'> 11/9 to 11/10 11/10
old code 11/9 to 11/10 11/10 11/10/24
25930 <re.Match object; span=(5, 7), match='to'> 2/23 to 3/2 3/2
old code 2/23 to 3/2 3/2 3/2/25
25931 <re.Match object; span=(5, 7), match='to'

27629 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27630 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27631 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27632 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27633 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27634 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27635 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27636 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27637 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
27638 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 t

29193 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29194 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29195 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29196 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29197 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29198 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29199 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29200 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29201 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
29202 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 t

In [765]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,event_date
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,18-Feb-98,SGP,2.5,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,18-Feb-98,SGP,-1.9,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,2002,SGP,None,Sprint,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,2003,SGP,None,Sprint,Male,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,25-Feb-92,SGP,None,Throw,Male,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31094,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International,27-Apr-25
31095,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International,27-Apr-25
31096,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International,27-Apr-25
31097,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,None,SGP,None,None,Male,London Marathon 2025,27-Apr,2025,International,27-Apr-25


In [766]:
# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], format='mixed', dayfirst=False)

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')

competitors['event_month'] = competitors['event_date_dt'].dt.month


In [767]:
competitors[competitors['EVENT']=='100m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0
43,Marc Brian Louis,10.45,None,None,None,None,100m,None,None,None,...,Male,National Championships,1 - 4 May,2025,International,4 May/25,2025-05-04,13 days 01:24:23.934620,13.0,5.0
44,Tan Daryl,10.62,None,None,None,None,100m,None,None,None,...,Male,National Championships,1 - 4 May,2025,International,4 May/25,2025-05-04,13 days 01:24:23.934620,13.0,5.0
45,Marc Brian Louis,10.66,None,None,None,None,100m,None,None,None,...,Male,National Championships,1 - 4 May,2025,International,4 May/25,2025-05-04,13 days 01:24:23.934620,13.0,5.0
46,Tan Daryl,10.81,None,None,None,None,100m,None,None,None,...,Male,National Championships,1 - 4 May,2025,International,4 May/25,2025-05-04,13 days 01:24:23.934620,13.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,Tan Daryl,11.21,None,None,1,None,100m,None,None,None,...,Male,Ashford AC Summer Open Series,8-Aug,2024,International,8-Aug-24,2024-08-08,282 days 01:24:23.934620,282.0,8.0
414,Joshua Chua Hanwei,10.99,None,None,1,None,100m,None,None,None,...,Male,HVC/AVT Meet #4,8-Mar,2025,International,8-Mar-25,2025-03-08,70 days 01:24:23.934620,70.0,3.0
415,Reuben Lee Siong En,10.86,None,None,5,None,100m,None,None,None,...,Male,Hong Kong Athletics Series 2,9 - 10 March,2024,International,10 March 24,2024-03-10,433 days 01:24:23.934620,433.0,3.0
416,Reuben Lee Siong En,10.9,None,None,8,None,100m,None,None,None,...,Male,2nd HKAAA Athletics Series Meeting,9 - 10 March,2024,International,10 March 24,2024-03-10,433 days 01:24:23.934620,433.0,3.0


In [768]:
'''
# Choose date range for WA qualification window

#competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

competitors=competitors.reset_index(drop=True)

start_1 = datetime.datetime(2023, 11, 5)
end_1 = datetime.datetime(2025, 5, 4)
start_2 = datetime.datetime(2024, 2, 25)
end_2 = datetime.datetime(2025, 8, 24)
start_3 = datetime.datetime(2024, 8, 1)
end_3 = datetime.datetime(2025, 8, 24)


start_date1 = np.datetime64(start_1)
end_date1 = np.datetime64(end_1)
start_date2 = np.datetime64(start_2)
end_date2 = np.datetime64(end_2)
start_date3 = np.datetime64(start_3)
end_date3 = np.datetime64(end_3)


mass = competitors['EVENT']==(('Marathon') or ('35km Racewalk'))

mask1 = ((competitors['EVENT']==(('Marathon') or ('35km Racewalk'))) & (competitors['event_date_dt'] >= start_date1) & (competitors['event_date_dt'] <= end_date1))
mask2 = ((competitors['EVENT']==(('10000m') or ('20km Racewalk') or ('Combined'))) & (competitors['event_date_dt'] >= start_date2) & (competitors['event_date_dt'] <= end_date2))
mask3 = ((competitors['EVENT']!=(('Marathon') or ('35km Racewalk') or ('10000m')|('20km Racewalk')|('Combined'))) & (competitors['event_date_dt'] >= start_date3) & (competitors['event_date_dt'] <= end_date3))

combined_mask = (mask1|mask2|mask3)

athletes_selected = competitors.loc[combined_mask]
'''

"\n# Choose date range for WA qualification window\n\n#competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]\n\ncompetitors=competitors.reset_index(drop=True)\n\nstart_1 = datetime.datetime(2023, 11, 5)\nend_1 = datetime.datetime(2025, 5, 4)\nstart_2 = datetime.datetime(2024, 2, 25)\nend_2 = datetime.datetime(2025, 8, 24)\nstart_3 = datetime.datetime(2024, 8, 1)\nend_3 = datetime.datetime(2025, 8, 24)\n\n\nstart_date1 = np.datetime64(start_1)\nend_date1 = np.datetime64(end_1)\nstart_date2 = np.datetime64(start_2)\nend_date2 = np.datetime64(end_2)\nstart_date3 = np.datetime64(start_3)\nend_date3 = np.datetime64(end_3)\n\n\nmass = competitors['EVENT']==(('Marathon') or ('35km Racewalk'))\n\nmask1 = ((competitors['EVENT']==(('Marathon') or ('35km Racewalk'))) & (competitors['event_date_dt'] >= start_date1) & (competitors['event_date_dt'] <= end_date1))\nmask2 = ((competitors['EVENT']==(('10000m') or ('20km Racewalk') or ('Combined'))) & (co

In [769]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/World Athletics Champs/')


competitors.to_csv('competitors_WA.csv', encoding='utf-8')

In [770]:
#athletes_selected = competitors[(competitors['YEAR']=='2024')|(competitors['YEAR']=='2025')]

#athletes_selected = competitors[(competitors['YEAR']=='2025')]

In [771]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,Female,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,Male,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,Male,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31094,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,Male,London Marathon 2025,27-Apr,2025,International,27-Apr-25,2025-04-27,20 days 01:24:23.934620,20.0,4.0
31095,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,Male,London Marathon 2025,27-Apr,2025,International,27-Apr-25,2025-04-27,20 days 01:24:23.934620,20.0,4.0
31096,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,Male,London Marathon 2025,27-Apr,2025,International,27-Apr-25,2025-04-27,20 days 01:24:23.934620,20.0,4.0
31097,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,Male,London Marathon 2025,27-Apr,2025,International,27-Apr-25,2025-04-27,20 days 01:24:23.934620,20.0,4.0


In [772]:
competitors[competitors['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
41,Jun Jie Calvin Quek,51.64,None,None,6.0,None,400m Hurdles,None,None,None,...,Male,12th Kinami Michitaka Memorial Athletics Meet,11-May,2025,International,11-May-25,2025-05-11,6 days 01:24:23.934620,6.0,5.0
139,Jun Jie Calvin Quek,10.86,None,None,2,None,100m,None,None,None,...,Male,Potch Invitational Meet,16-Apr,2025,International,16-Apr-25,2025-04-16,31 days 01:24:23.934620,31.0,4.0
6254,Jun Jie Calvin Quek,51.8,None,None,1,None,400m Hurdles,None,None,None,...,Male,Taiwan Athletics Open,1 - 2 June,2024,International,2 June/24,2024-06-02,349 days 01:24:23.934620,349.0,6.0
6256,Jun Jie Calvin Quek,52.05,None,None,8,None,400m Hurdles,None,None,None,...,Male,11th Kinami Michitaka Memorial Athletics Meet,12-May,2024,International,12-May-24,2024-05-12,370 days 01:24:23.934620,370.0,5.0
6328,Jun Jie Calvin Quek,52.62,None,None,9,None,400m Hurdles,None,None,None,...,Male,Sydney Track Classic,15-Mar,2025,International,15-Mar-25,2025-03-15,63 days 01:24:23.934620,63.0,3.0
6329,Jun Jie Calvin Quek,50.77,None,None,1,None,400m Hurdles,None,None,None,...,Male,Potch Invitational Meet,16-Apr,2025,International,16-Apr-25,2025-04-16,31 days 01:24:23.934620,31.0,4.0
6359,Jun Jie Calvin Quek,51.79,None,None,2,None,400m Hurdles,None,None,None,...,Male,XXXIII Qosanov Memorial,22 - 23 June,2024,International,23 June 24,2024-06-23,328 days 01:24:23.934620,328.0,6.0
6360,Jun Jie Calvin Quek,56.95,None,None,3,None,400m Hurdles,None,None,None,...,Male,XXXIII Qosanov Memorial,22 - 23 June,2024,International,23 June 24,2024-06-23,328 days 01:24:23.934620,328.0,6.0
6361,Jun Jie Calvin Quek,52.18,None,None,3,None,400m Hurdles,None,None,None,...,Male,Second Belt and Road Invitational Athletics Me...,27 - 29 May,2024,International,29 May 24,2024-05-29,353 days 01:24:23.934620,353.0,5.0
6362,Jun Jie Calvin Quek,51.33,None,None,2,None,400m Hurdles,None,None,None,...,Male,Second Belt and Road Invitational Athletics Me...,27 - 29 May,2024,International,29 May 24,2024-05-29,353 days 01:24:23.934620,353.0,5.0


In [773]:
# Wind mapping

#mask = athletes['EVENT'].str.contains(r'\d{1}.\d{1}', na=True, regex=True)
#athletes.loc[mask, 'WIND_MAP'] = '400m'


In [774]:
athletes=competitors

In [775]:
# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()


# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

#mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open|Invitational', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('None|0.762|0.914', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='4kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7.26', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'7.26kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'4.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'2kg|2.00kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'3000m Steeplechase|3000m S\/C', na=True) & athletes['REGION'].str.contains(r'International', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase|S\/C', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'3000m Steeplechase|3000m S\/C', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



In [776]:
competitors=athletes

In [777]:
competitors[(competitors['MAPPED_EVENT']=='Decathlon')]

#athletes[(athletes['EVENT']=='110m Hurdles')].tail(50)

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
17170,Lucas Le Cong Fun,6431,None,None,1,U20,Decathlon,None,None,None,...,National U20,17 - 18 February,2024,International,18 February 24,2024-02-18,454 days 01:24:23.934620,454.0,2.0,Decathlon
17171,Jayden Ng,5716,None,None,3,None,Decathlon U18,None,None,None,...,National U20,22 - 23 March,2025,International,23 March 25,2025-03-23,55 days 01:24:23.934620,55.0,3.0,Decathlon


In [778]:
for col in competitors.columns:
    competitors[col] = competitors[col].astype(str)
    competitors[col] = competitors[col].str.replace('\xa0', ' ', regex=True)
    competitors[col] = competitors[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    competitors[col] = competitors[col].str.replace('\r', ' ', regex=True)
    competitors[col] = competitors[col].str.replace('\n', ' ', regex=True)
    competitors[col] = competitors[col].str.strip()



In [779]:
# Choose date range for WA qualification window

#competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

competitors=competitors.reset_index(drop=True)

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date_dt']) # convert to datetime again


start_1 = datetime.datetime(2023, 11, 5)
end_1 = datetime.datetime(2025, 5, 4)

start_2 = datetime.datetime(2024, 2, 25)
end_2 = datetime.datetime(2025, 8, 24)

start_3 = datetime.datetime(2024, 8, 1)
#start_3 = datetime.datetime(2025, 5, 1)
end_3 = datetime.datetime(2025, 8, 24)


start_date1 = np.datetime64(start_1)
end_date1 = np.datetime64(end_1)
start_date2 = np.datetime64(start_2)
end_date2 = np.datetime64(end_2)
start_date3 = np.datetime64(start_3)
end_date3 = np.datetime64(end_3)


mass = competitors['MAPPED_EVENT']==(('Marathon') or ('35km Racewalk'))

mask1 = ((competitors['MAPPED_EVENT']==(('Marathon') or ('35km Racewalk'))) & (competitors['event_date_dt'] >= start_date1) & (competitors['event_date_dt'] <= end_date1))
mask2 = ((competitors['MAPPED_EVENT']==(('10,000m') or ('20km Racewalk') or ('Combined'))) & (competitors['event_date_dt'] >= start_date2) & (competitors['event_date_dt'] <= end_date2))
mask3 = ((competitors['MAPPED_EVENT']!=(('Marathon') or ('35km Racewalk') or ('10,000m')|('20km Racewalk')|('Combined'))) & (competitors['event_date_dt'] >= start_date3) & (competitors['event_date_dt'] <= end_date3))

combined_mask = (mask1|mask2|mask3)

athletes_selected = competitors.loc[combined_mask]


In [780]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/World Athletics Champs/')


athletes_selected.to_csv('athletes_post_map_WA.csv', sep=',', encoding='utf-8-sig', index=False)


In [781]:
#credentials = service_account.Credentials.from_service_account_file(
#    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
#)

#sql="""
#SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
#FROM `saa-analytics.results.saa_full`
#WHERE STAGE='Final' AND COMPETITION='SEA Game AND RANK='3'
#"""

#benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




In [782]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/World Athletics Champs/')

wac_benchmarks = pd.read_csv("2025 World Athletics Benchmarks.csv")


In [783]:
wac_benchmarks

,EVENT,GENDER,BENCHMARK
0,100m,Male,10
1,100m,Female,11.07
2,200m,Male,20.16
3,200m,Female,22.57
4,400m,Male,44.85
5,400m,Female,50.75
6,800m,Male,01:44.5
7,800m,Female,01:59.0
8,1500m,Male,03:33.0
9,1500m,Female,04:01.5


In [784]:
for col in wac_benchmarks.columns:
    wac_benchmarks[col] = wac_benchmarks[col].astype(str)
    wac_benchmarks[col] = wac_benchmarks[col].str.replace('\xa0', ' ', regex=True)
    wac_benchmarks[col] = wac_benchmarks[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    wac_benchmarks[col] = wac_benchmarks[col].str.replace('\r', ' ', regex=True)
    wac_benchmarks[col] = wac_benchmarks[col].str.replace('\n', ' ', regex=True)
    wac_benchmarks[col] = wac_benchmarks[col].str.strip()


In [785]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                elif '5000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    

                    
                elif '1500m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                    
                    if len(substring)==7:  # format is X:XX:XX and not XX:XX:XX 
                        
                        idx = 4 # 5th character position
                        replacement = "."
                        metric = '0' + metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                        
                    else:  # format is XX:XX:XX
                        
                        idx = 5 # 5th character position
                        replacement = "."
                        metric = metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())  
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [786]:
def process_benchmarks(df):
    
    for i in range(len(df)):

        rowIndex = df.index[i]

        input_string=df.iloc[rowIndex,0]
    
        metric=df.iloc[rowIndex,2]
    
        if metric==None:
        
            continue
        
        out = convert_time(i, input_string, metric)
        
        print(rowIndex, input_string, out)

    
        df.loc[rowIndex, 'Metric'] = out
    
    return df

In [787]:
process_benchmarks(wac_benchmarks)

0 100m 10.0
1 100m 11.07
2 200m 20.16
3 200m 22.57
4 400m 44.85
5 400m 50.75
6 800m 104.5
7 800m 119.0
8 1500m 213.0
9 1500m 241.5
10 Mile 230.0
11 Mile 259.9
12 5000m 781.0
13 5000m 890.0
14 3000m Steeplechase 495.0
15 3000m Steeplechase 558.0
16 10,000m 1859.2
17 10,000m 2038.9
18 110m Hurdles 13.27
19 100m Hurdles 12.73
20 400m Hurdles 48.5
21 400m Hurdles 54.65
22 High Jump 2.33
23 High Jump 1.97
24 Long Jump 8.27
25 Long Jump 6.86
26 Triple Jump 17.22
27 Triple Jump 14.55
28 Shot Put 21.5
29 Shot Put 18.8
30 Discus Throw 67.5
31 Discus Throw 64.5
32 Javelin Throw 85.5
33 Javelin Throw 64.0
34 Hammer Throw 78.2
35 Hammer Throw 74.0
36 Pole Vault 5.34
37 Pole Vault 4.03
38 Decathlon 8500.0
39 Heptathlon 6500.0
40 Marathon 7590.0
41 Marathon 8610.0
42 20km Racewalk 4760.0
43 20km Racewalk 5340.0
44 35km Racewalk 8880.0
45 35km Racewalk 10080.0


,EVENT,GENDER,BENCHMARK,Metric
0,100m,Male,10,10.00
1,100m,Female,11.07,11.07
2,200m,Male,20.16,20.16
3,200m,Female,22.57,22.57
4,400m,Male,44.85,44.85
5,400m,Female,50.75,50.75
6,800m,Male,01:44.5,104.50
7,800m,Female,01:59.0,119.00
8,1500m,Male,03:33.0,213.00
9,1500m,Female,04:01.5,241.50


In [788]:
for i in range(len(wac_benchmarks)):
        
    rowIndex = wac_benchmarks.index[i]

    input_string=wac_benchmarks.iloc[rowIndex,0]
    
    metric=wac_benchmarks.iloc[rowIndex,2]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    wac_benchmarks.loc[rowIndex, 'Metric'] = out

0 100m 10.0
1 100m 11.07
2 200m 20.16
3 200m 22.57
4 400m 44.85
5 400m 50.75
6 800m 104.5
7 800m 119.0
8 1500m 213.0
9 1500m 241.5
10 Mile 230.0
11 Mile 259.9
12 5000m 781.0
13 5000m 890.0
14 3000m Steeplechase 495.0
15 3000m Steeplechase 558.0
16 10,000m 1859.2
17 10,000m 2038.9
18 110m Hurdles 13.27
19 100m Hurdles 12.73
20 400m Hurdles 48.5
21 400m Hurdles 54.65
22 High Jump 2.33
23 High Jump 1.97
24 Long Jump 8.27
25 Long Jump 6.86
26 Triple Jump 17.22
27 Triple Jump 14.55
28 Shot Put 21.5
29 Shot Put 18.8
30 Discus Throw 67.5
31 Discus Throw 64.5
32 Javelin Throw 85.5
33 Javelin Throw 64.0
34 Hammer Throw 78.2
35 Hammer Throw 74.0
36 Pole Vault 5.34
37 Pole Vault 4.03
38 Decathlon 8500.0
39 Heptathlon 6500.0
40 Marathon 7590.0
41 Marathon 8610.0
42 20km Racewalk 4760.0
43 20km Racewalk 5340.0
44 35km Racewalk 8880.0
45 35km Racewalk 10080.0


In [789]:
wac_benchmarks

,EVENT,GENDER,BENCHMARK,Metric
0,100m,Male,10,10.00
1,100m,Female,11.07,11.07
2,200m,Male,20.16,20.16
3,200m,Female,22.57,22.57
4,400m,Male,44.85,44.85
5,400m,Female,50.75,50.75
6,800m,Male,01:44.5,104.50
7,800m,Female,01:59.0,119.00
8,1500m,Male,03:33.0,213.00
9,1500m,Female,04:01.5,241.50


In [790]:
mask = wac_benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put|Decathlon|Heptathlon', na=True)

wac_benchmarks.loc[mask, '2%']=wac_benchmarks['Metric']*0.98
wac_benchmarks.loc[mask, '3.5%']=wac_benchmarks['Metric']*0.965
wac_benchmarks.loc[mask, '5%']=wac_benchmarks['Metric']*0.95

wac_benchmarks.loc[~mask, '2%']=wac_benchmarks['Metric']*1.02
wac_benchmarks.loc[~mask, '3.5%']=wac_benchmarks['Metric']*1.035
wac_benchmarks.loc[~mask, '5%']=wac_benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [791]:
wac_benchmarks

,EVENT,GENDER,BENCHMARK,Metric,2%,3.5%,5%
0,100m,Male,10,10.00,10.2000,10.35000,10.5000
1,100m,Female,11.07,11.07,11.2914,11.45745,11.6235
2,200m,Male,20.16,20.16,20.5632,20.86560,21.1680
3,200m,Female,22.57,22.57,23.0214,23.35995,23.6985
4,400m,Male,44.85,44.85,45.7470,46.41975,47.0925
5,400m,Female,50.75,50.75,51.7650,52.52625,53.2875
6,800m,Male,01:44.5,104.50,106.5900,108.15750,109.7250
7,800m,Female,01:59.0,119.00,121.3800,123.16500,124.9500
8,1500m,Male,03:33.0,213.00,217.2600,220.45500,223.6500
9,1500m,Female,04:01.5,241.50,246.3300,249.95250,253.5750


In [792]:
wac_benchmarks.to_csv('wac_benchmarks_processed.csv', encoding='utf-8')

In [793]:
athletes_selected

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0,100m
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0,200m
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0,400m
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0,400m
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,Dubai World Para Athletics Grand Prix - 16th F...,10 - 13 February,2025,International,13 February 25,2025-02-13,93 days 01:24:23.934620,93.0,2.0,Shot Put
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31094,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,London Marathon 2025,27-Apr,2025,International,27-Apr-25,2025-04-27,20 days 01:24:23.934620,20.0,4.0,Marathon
31095,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,London Marathon 2025,27-Apr,2025,International,27-Apr-25,2025-04-27,20 days 01:24:23.934620,20.0,4.0,Marathon
31096,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,London Marathon 2025,27-Apr,2025,International,27-Apr-25,2025-04-27,20 days 01:24:23.934620,20.0,4.0,Marathon
31097,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,London Marathon 2025,27-Apr,2025,International,27-Apr-25,2025-04-27,20 days 01:24:23.934620,20.0,4.0,Marathon


In [794]:
athletes_selected[athletes_selected['NAME']=='Heng, Richard']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
26452,"Heng, Richard",35:36.1,National University Singapore,22,4,Open,Run,10000,,nan,...,SA Allcomers Meet 2,2/25 to 3/3,2024,Local,3/3/24,2024-03-03,440 days 01:24:23.934620,440.0,3.0,"10,000m"
28105,"Heng, Richard",34:11.7,Club Zoom,22,2,Open,Run,10000,,nan,...,84th Singapore Open Track & Field,4/18 to 4/19,2024,Local,4/19/24,2024-04-19,393 days 01:24:23.934620,393.0,4.0,"10,000m"


In [795]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [796]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes_selected, 
    right=wac_benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['EVENT', 'GENDER'],
)

In [797]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,200m,200m,22.57,22.57,23.0214,23.35995,23.6985
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,400m,400m,50.75,50.75,51.7650,52.52625,53.2875
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,400m,400m,44.85,44.85,45.7470,46.41975,47.0925
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,Shot Put,Shot Put,21.5,21.50,21.0700,20.74750,20.4250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,20 days 01:24:23.934620,20.0,4.0,Marathon,Marathon,2:06:30,7590.00,7741.8000,7855.65000,7969.5000
14988,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,20 days 01:24:23.934620,20.0,4.0,Marathon,Marathon,2:06:30,7590.00,7741.8000,7855.65000,7969.5000
14989,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,20 days 01:24:23.934620,20.0,4.0,Marathon,Marathon,2:06:30,7590.00,7741.8000,7855.65000,7969.5000
14990,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,20 days 01:24:23.934620,20.0,4.0,Marathon,Marathon,2:06:30,7590.00,7741.8000,7855.65000,7969.5000


In [798]:
df[df['NAME']=='Shanti Veronica Pereira']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
75,Shanti Veronica Pereira,11.63,None,None,7,None,100m,None,None,None,...,279 days 01:24:23.934620,279.0,8.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
98,Shanti Veronica Pereira,11.52,None,None,6,None,100m,None,None,None,...,63 days 01:24:23.934620,63.0,3.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
99,Shanti Veronica Pereira,11.67,None,None,1,None,100m,None,None,None,...,31 days 01:24:23.934620,31.0,4.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
102,Shanti Veronica Pereira,11.64,None,None,1,None,100m,None,None,None,...,273 days 01:24:23.934620,273.0,8.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
103,Shanti Veronica Pereira,11.77,None,None,2,None,100m,None,None,None,...,273 days 01:24:23.934620,273.0,8.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
246,Shanti Veronica Pereira,11.68,None,None,3,None,100m,None,None,None,...,84 days 01:24:23.934620,84.0,2.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
315,Shanti Veronica Pereira,11.67w,None,None,3,None,100m,None,None,None,...,18 days 01:24:23.934620,18.0,4.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
316,Shanti Veronica Pereira,11.54,None,None,3,None,100m,None,None,None,...,18 days 01:24:23.934620,18.0,4.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
321,Shanti Veronica Pereira,11.58,None,None,6,None,100m,None,None,None,...,256 days 01:24:23.934620,256.0,9.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
322,Shanti Veronica Pereira,11.53,None,None,3,None,100m,None,None,None,...,256 days 01:24:23.934620,256.0,9.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235


In [799]:
df[df['MAPPED_EVENT']=='100m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
17,Marc Brian Louis,10.45,None,None,None,None,100m,None,None,None,...,13 days 01:24:23.934620,13.0,5.0,100m,100m,10,10.00,10.2000,10.35000,10.5000
18,Tan Daryl,10.62,None,None,None,None,100m,None,None,None,...,13 days 01:24:23.934620,13.0,5.0,100m,100m,10,10.00,10.2000,10.35000,10.5000
19,Marc Brian Louis,10.66,None,None,None,None,100m,None,None,None,...,13 days 01:24:23.934620,13.0,5.0,100m,100m,10,10.00,10.2000,10.35000,10.5000
20,Tan Daryl,10.81,None,None,None,None,100m,None,None,None,...,13 days 01:24:23.934620,13.0,5.0,100m,100m,10,10.00,10.2000,10.35000,10.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8228,"NEO, EN YU",13.48,Singapore Sports School,17,4,U18,Dash,100,None,None,...,251 days 01:24:23.934620,251.0,9.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
8232,"Tong, Zong Wei",11.68,Hwa Chong Alumni Association,17,3,U18,Dash,100,None,None,...,251 days 01:24:23.934620,251.0,9.0,100m,100m,10,10.00,10.2000,10.35000,10.5000
8233,"Curran, Harry Irfan",11.27,Team Cicada Trackers,16,1,U18,Dash,100,None,None,...,251 days 01:24:23.934620,251.0,9.0,100m,100m,10,10.00,10.2000,10.35000,10.5000
8235,"MUHAMMAD RASHID, EMIR",11.64,Singapore Sports School,17,2,U18,Dash,100,None,None,...,251 days 01:24:23.934620,251.0,9.0,100m,100m,10,10.00,10.2000,10.35000,10.5000


In [800]:
# replace '-' with NaN

df['RESULT'] = df['RESULT'].replace(regex=r'–', value=np.NaN)
#df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [801]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,100m,100m,11.07,11.07,11.2914,11.45745,11.6235
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,200m,200m,22.57,22.57,23.0214,23.35995,23.6985
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,400m,400m,50.75,50.75,51.7650,52.52625,53.2875
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,400m,400m,44.85,44.85,45.7470,46.41975,47.0925
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,93 days 01:24:23.934620,93.0,2.0,Shot Put,Shot Put,21.5,21.50,21.0700,20.74750,20.4250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,20 days 01:24:23.934620,20.0,4.0,Marathon,Marathon,2:06:30,7590.00,7741.8000,7855.65000,7969.5000
14988,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,20 days 01:24:23.934620,20.0,4.0,Marathon,Marathon,2:06:30,7590.00,7741.8000,7855.65000,7969.5000
14989,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,20 days 01:24:23.934620,20.0,4.0,Marathon,Marathon,2:06:30,7590.00,7741.8000,7855.65000,7969.5000
14990,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,20 days 01:24:23.934620,20.0,4.0,Marathon,Marathon,2:06:30,7590.00,7741.8000,7855.65000,7969.5000


In [802]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
11061,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,...,391 days 01:24:23.934620,391.0,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5


In [803]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/World Athletics Champs/')


df.to_csv('WAC_postmap.csv', sep=',', encoding='utf-8-sig', index=False)


In [804]:
# Convert results and seed into seconds format

df.reset_index(drop=True, inplace=True)

for col in df.columns:
    
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('\xa0', ' ', regex=True)
    df[col] = df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df[col] = df[col].str.replace('\r', ' ', regex=True)
    df[col] = df[col].str.replace('\n', ' ', regex=True)
    df[col] = df[col].str.strip()

for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    event=df.loc[rowIndex,'MAPPED_EVENT']    # event description
    
    result=df.loc[rowIndex,'RESULT'] # result
    
    if result=='—' or result=='DQ' or result=='SCR' or result=='FS' or result=='DNQ' or result=='DNS' or result=='NH' or result=='NM' or result=='FOUL' or result=='DNF' or result=='SR' :
        continue
    
    result_out = convert_time(i, event, result)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out



In [805]:
#df["AGE"].fillna(0, inplace=True)
#df['AGE'] = df['AGE'].astype('float')

In [806]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,93.0,2.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,93.0,2.0,200m,200m,22.57,22.57,23.0214,23.359949999999998,23.698500000000003,37.98
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,93.0,2.0,400m,400m,50.75,50.75,51.765,52.52625,53.2875,70.3
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,93.0,2.0,400m,400m,44.85,44.85,45.747,46.41975,47.0925,59.18
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,93.0,2.0,Shot Put,Shot Put,21.5,21.5,21.07,20.7475,20.425,8.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,20.0,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5,14964.0
14988,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,20.0,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5,17196.0
14989,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,20.0,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5,21159.0
14990,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,20.0,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5,16595.0


In [807]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [808]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,MAPPED_EVENT,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,2.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,,
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,2.0,200m,200m,22.57,22.57,23.0214,23.359949999999998,23.698500000000003,37.98,37.98
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,2.0,400m,400m,50.75,50.75,51.765,52.52625,53.2875,70.3,70.3
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,2.0,400m,400m,44.85,44.85,45.747,46.41975,47.0925,59.18,59.18
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,2.0,Shot Put,Shot Put,21.5,21.5,21.07,20.7475,20.425,8.36,8.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5,14964.0,14964.0
14988,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5,17196.0,17196.0
14989,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5,21159.0,21159.0
14990,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,4.0,Marathon,Marathon,2:06:30,7590.0,7741.8,7855.65,7969.5,16595.0,16595.0


In [809]:
df[df['NAME']=='Shanti Veronica Pereira']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,MAPPED_EVENT,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
75,Shanti Veronica Pereira,11.63,None,None,7,None,100m,None,None,None,...,8.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.63,11.63
98,Shanti Veronica Pereira,11.52,None,None,6,None,100m,None,None,None,...,3.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.52,11.52
99,Shanti Veronica Pereira,11.67,None,None,1,None,100m,None,None,None,...,4.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.67,11.67
102,Shanti Veronica Pereira,11.64,None,None,1,None,100m,None,None,None,...,8.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.64,11.64
103,Shanti Veronica Pereira,11.77,None,None,2,None,100m,None,None,None,...,8.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.77,11.77
246,Shanti Veronica Pereira,11.68,None,None,3,None,100m,None,None,None,...,2.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.68,11.68
315,Shanti Veronica Pereira,11.67w,None,None,3,None,100m,None,None,None,...,4.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,,
316,Shanti Veronica Pereira,11.54,None,None,3,None,100m,None,None,None,...,4.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.54,11.54
321,Shanti Veronica Pereira,11.58,None,None,6,None,100m,None,None,None,...,9.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.58,11.58
322,Shanti Veronica Pereira,11.53,None,None,3,None,100m,None,None,None,...,9.0,100m,100m,11.07,11.07,11.291400000000001,11.45745,11.6235,11.53,11.53


In [810]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']] = df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']].apply(pd.to_numeric, errors='coerce')

In [811]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw|Decathlon|Heptathlon|decathlon|heptathlon', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
#df=df.loc[df['COMPETITION']!='Southeast Asian Games'] # Do not include results from SEAG in dataset

In [812]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [813]:
df[df['NAME']=='Shanti Veronica Pereira']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
75,Shanti Veronica Pereira,11.63,None,None,7,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.63,11.63,-0.3386,-0.17255,-0.0065,-0.56,-0.058717
98,Shanti Veronica Pereira,11.52,None,None,6,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.52,11.52,-0.2286,-0.06255,0.1035,-0.45,0.934959
99,Shanti Veronica Pereira,11.67,None,None,1,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.67,11.67,-0.3786,-0.21255,-0.0465,-0.60,-0.420054
102,Shanti Veronica Pereira,11.64,None,None,1,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.64,11.64,-0.3486,-0.18255,-0.0165,-0.57,-0.149051
103,Shanti Veronica Pereira,11.77,None,None,2,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.77,11.77,-0.4786,-0.31255,-0.1465,-0.70,-1.323397
246,Shanti Veronica Pereira,11.68,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.68,11.68,-0.3886,-0.22255,-0.0565,-0.61,-0.510388
315,Shanti Veronica Pereira,11.67w,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,,NaN,NaN,NaN,NaN,NaN,NaN
316,Shanti Veronica Pereira,11.54,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.54,11.54,-0.2486,-0.08255,0.0835,-0.47,0.754291
321,Shanti Veronica Pereira,11.58,None,None,6,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.58,11.58,-0.2886,-0.12255,0.0435,-0.51,0.392954
322,Shanti Veronica Pereira,11.53,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.53,11.53,-0.2386,-0.07255,0.0935,-0.46,0.844625


In [814]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/World Athletics Champs/')


df.to_csv('WAC_postmap_benchmarked.csv', sep=',', encoding='utf-8-sig', index=False)


In [815]:
df[df['NAME']=='Shanti Veronica Pereira']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
75,Shanti Veronica Pereira,11.63,None,None,7,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.63,11.63,-0.3386,-0.17255,-0.0065,-0.56,-0.058717
98,Shanti Veronica Pereira,11.52,None,None,6,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.52,11.52,-0.2286,-0.06255,0.1035,-0.45,0.934959
99,Shanti Veronica Pereira,11.67,None,None,1,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.67,11.67,-0.3786,-0.21255,-0.0465,-0.60,-0.420054
102,Shanti Veronica Pereira,11.64,None,None,1,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.64,11.64,-0.3486,-0.18255,-0.0165,-0.57,-0.149051
103,Shanti Veronica Pereira,11.77,None,None,2,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.77,11.77,-0.4786,-0.31255,-0.1465,-0.70,-1.323397
246,Shanti Veronica Pereira,11.68,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.68,11.68,-0.3886,-0.22255,-0.0565,-0.61,-0.510388
315,Shanti Veronica Pereira,11.67w,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,,NaN,NaN,NaN,NaN,NaN,NaN
316,Shanti Veronica Pereira,11.54,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.54,11.54,-0.2486,-0.08255,0.0835,-0.47,0.754291
321,Shanti Veronica Pereira,11.58,None,None,6,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.58,11.58,-0.2886,-0.12255,0.0435,-0.51,0.392954
322,Shanti Veronica Pereira,11.53,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.53,11.53,-0.2386,-0.07255,0.0935,-0.46,0.844625


In [816]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,11.2914,11.45745,11.6235,,NaN,NaN,NaN,NaN,NaN,NaN
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,23.0214,23.35995,23.6985,37.98,37.98,-14.9586,-14.62005,-14.2815,-15.41,-63.276473
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,51.7650,52.52625,53.2875,70.3,70.30,-18.5350,-17.77375,-17.0125,-19.55,-33.522167
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,45.7470,46.41975,47.0925,59.18,59.18,-13.4330,-12.76025,-12.0875,-14.33,-26.950948
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,21.0700,20.74750,20.4250,8.36,8.36,-12.7100,-12.38750,-12.0650,-13.14,-56.116279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,14964.0,14964.00,-7222.2000,-7108.35000,-6994.5000,-7374.00,-92.154150
14988,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,17196.0,17196.00,-9454.2000,-9340.35000,-9226.5000,-9606.00,-121.561265
14989,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,21159.0,21159.00,-13417.2000,-13303.35000,-13189.5000,-13569.00,-173.774704
14990,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,16595.0,16595.00,-8853.2000,-8739.35000,-8625.5000,-9005.00,-113.642951


In [817]:

# Read a variation name list and corrections from CSVs
'''
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

for index, row in names.iterrows():
        
    print(names.VARIATION, names.NAME)
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")
'''

'\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/\')\n\nnames = pd.read_csv("name_variations.csv")\n\nfor index, row in names.iterrows():\n        \n    print(names.VARIATION, names.NAME)\n    df[\'NAME\'] = df[\'NAME\'].replace(regex=rf"{row[\'VARIATION\']}", value=f"{row[\'NAME\']}")\n'

In [818]:
# Read name variations from GCS name lists bucket (Still in beta)


df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})

# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")


In [819]:
df_names=df[((df['NAME']=='Shanti Veronica Pereira')|(df['NAME']=='Marican, Shohib'))]

In [820]:
df_names

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
12663,"Marican, Shohib",04:16.8,Macritchie Runners 25,27,8,Open,Run,1500,,S654797,...,217.26,220.455,223.65,256.8,256.8,-39.54,-36.345,-33.15,-43.8,-15.563380
12686,"Marican, Shohib",16:52.2,Macritchie Runners 25,27,8,Open,Run,5000,,S654797,...,796.62,808.335,820.05,1012.2,1012.2,-215.58,-203.865,-192.15,-231.2,-24.603073
13831,"Marican, Shohib",04:15.5,Macritchie Runners 25,28,12,Open,Run,1500,None,S547E97,...,217.26,220.455,223.65,255.5,255.5,-38.24,-35.045,-31.85,-42.5,-14.953052


In [821]:
df_names.to_csv("names.csv", encoding='utf-8')

In [822]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

#df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 

df_select = df[(df['TEAM']!='Malaysia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='China')&(df['TEAM']!='Thailand') 
                       &(df['TEAM']!='South Korea')&(df['TEAM']!='Laos')&(df['TEAM']!='Myanmar') 
                       &(df['TEAM']!='Philippines')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Chinese Taipei')&(df['TEAM']!='Gurkha Contingent') 
                       &(df['TEAM']!='Australia')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Hong Kong')&(df['TEAM']!='PERAK')&(df['TEAM']!='Sri Lanka') 
                       &(df['TEAM']!='Indonesia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='MALAYSIA') 
                       &(df['TEAM']!='PHILIPPINES') & (df['TEAM']!='SOUTH KOREA')&(df['TEAM']!='Waseda') 
                       &(df['TEAM']!='LAOS')&(df['TEAM']!='CHINESE TAIPEI')&(df['TEAM']!='Vietnam')
                       &(df['TEAM']!='INDIA')&(df['TEAM']!='Hong Kong, China')&(df['TEAM']!='AIC JAPAN')] 

In [823]:
df_select

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,11.2914,11.45745,11.6235,,NaN,NaN,NaN,NaN,NaN,NaN
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,23.0214,23.35995,23.6985,37.98,37.98,-14.9586,-14.62005,-14.2815,-15.41,-63.276473
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,51.7650,52.52625,53.2875,70.3,70.30,-18.5350,-17.77375,-17.0125,-19.55,-33.522167
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,45.7470,46.41975,47.0925,59.18,59.18,-13.4330,-12.76025,-12.0875,-14.33,-26.950948
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,21.0700,20.74750,20.4250,8.36,8.36,-12.7100,-12.38750,-12.0650,-13.14,-56.116279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,14964.0,14964.00,-7222.2000,-7108.35000,-6994.5000,-7374.00,-92.154150
14988,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,17196.0,17196.00,-9454.2000,-9340.35000,-9226.5000,-9606.00,-121.561265
14989,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,21159.0,21159.00,-13417.2000,-13303.35000,-13189.5000,-13569.00,-173.774704
14990,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,16595.0,16595.00,-8853.2000,-8739.35000,-8625.5000,-9005.00,-113.642951


In [824]:
df_select[df_select['NAME']=='PEREIRA, VERONICA SHANTI']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
75,"PEREIRA, VERONICA SHANTI",11.63,None,None,7,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.63,11.63,-0.3386,-0.17255,-0.0065,-0.56,-0.058717
98,"PEREIRA, VERONICA SHANTI",11.52,None,None,6,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.52,11.52,-0.2286,-0.06255,0.1035,-0.45,0.934959
99,"PEREIRA, VERONICA SHANTI",11.67,None,None,1,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.67,11.67,-0.3786,-0.21255,-0.0465,-0.60,-0.420054
102,"PEREIRA, VERONICA SHANTI",11.64,None,None,1,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.64,11.64,-0.3486,-0.18255,-0.0165,-0.57,-0.149051
103,"PEREIRA, VERONICA SHANTI",11.77,None,None,2,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.77,11.77,-0.4786,-0.31255,-0.1465,-0.70,-1.323397
246,"PEREIRA, VERONICA SHANTI",11.68,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.68,11.68,-0.3886,-0.22255,-0.0565,-0.61,-0.510388
315,"PEREIRA, VERONICA SHANTI",11.67w,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,,NaN,NaN,NaN,NaN,NaN,NaN
316,"PEREIRA, VERONICA SHANTI",11.54,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.54,11.54,-0.2486,-0.08255,0.0835,-0.47,0.754291
321,"PEREIRA, VERONICA SHANTI",11.58,None,None,6,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.58,11.58,-0.2886,-0.12255,0.0435,-0.51,0.392954
322,"PEREIRA, VERONICA SHANTI",11.53,None,None,3,None,100m,None,None,None,...,11.2914,11.45745,11.6235,11.53,11.53,-0.2386,-0.07255,0.0935,-0.46,0.844625


In [825]:
# Remove spurious SEED results 

#df_select=df_select[~((df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m'))]
#df_select=df_select[~((df_select['NAME']=='LOH, BEAVERLY') & (df_select['MAPPED_EVENT']=='400m'))]
#df_select=df_select[~((df_select['NAME']=='Vaz, Luisa Marie') & (df_select['MAPPED_EVENT']=='High jump'))]


In [826]:
df_select

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,11.2914,11.45745,11.6235,,NaN,NaN,NaN,NaN,NaN,NaN
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,23.0214,23.35995,23.6985,37.98,37.98,-14.9586,-14.62005,-14.2815,-15.41,-63.276473
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,51.7650,52.52625,53.2875,70.3,70.30,-18.5350,-17.77375,-17.0125,-19.55,-33.522167
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,45.7470,46.41975,47.0925,59.18,59.18,-13.4330,-12.76025,-12.0875,-14.33,-26.950948
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,21.0700,20.74750,20.4250,8.36,8.36,-12.7100,-12.38750,-12.0650,-13.14,-56.116279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,14964.0,14964.00,-7222.2000,-7108.35000,-6994.5000,-7374.00,-92.154150
14988,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,17196.0,17196.00,-9454.2000,-9340.35000,-9226.5000,-9606.00,-121.561265
14989,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,21159.0,21159.00,-13417.2000,-13303.35000,-13189.5000,-13569.00,-173.774704
14990,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,16595.0,16595.00,-8853.2000,-8739.35000,-8625.5000,-9005.00,-113.642951


In [827]:
'''
# Read list of foreigners

foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')
'''


"\n# Read list of foreigners\n\nforeigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')\n"

In [828]:
# Read list of foreigners from GCS bucket

file_path = "gs://name_lists/List of Foreigners.csv"
foreigners = pd.read_csv(file_path,
                 sep=",",
                 encoding="unicode escape",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


In [829]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
235,Kashama,Biwesa Daniel
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian


In [830]:
'''
def replace_special_chars_with_space(text):
    chars_to_replace = "	ï»¿"
    for char in chars_to_replace:
        text = text.replace(char, " ")
    return text
    
### Reasign column names    
foreigners.columns = [replace_special_chars_with_space(col) for col in foreigners.columns ]

'''

In [831]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
235,Kashama,Biwesa Daniel
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian


In [832]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [833]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [834]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [835]:
no_foreigners_list

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Maisarah Mohamed Hassan,16.46w,None,None,3,T36,100m,None,None,None,...,11.2914,11.45745,11.6235,,NaN,NaN,NaN,NaN,NaN,NaN
1,Maisarah Mohamed Hassan,37.98,None,None,5,T35/T36/T37,200m,None,None,None,...,23.0214,23.35995,23.6985,37.98,37.98,-14.9586,-14.62005,-14.2815,-15.41,-63.276473
2,Siti Nurhayati Ali Aksar,01:10.3,None,None,3,T20,400m,None,None,None,...,51.7650,52.52625,53.2875,70.3,70.30,-18.5350,-17.77375,-17.0125,-19.55,-33.522167
3,Maveric Dong En Lee,59.18,None,None,3,T20,400m,None,None,None,...,45.7470,46.41975,47.0925,59.18,59.18,-13.4330,-12.76025,-12.0875,-14.33,-26.950948
4,Muhammad Diroy Noordin,8.36,None,None,4,F40/F42/F46/F63,Shot Put,None,None,None,...,21.0700,20.74750,20.4250,8.36,8.36,-12.7100,-12.38750,-12.0650,-13.14,-56.116279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987,"Seth, Sandeep",04:09:24,None,None,14382,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,14964.0,14964.00,-7222.2000,-7108.35000,-6994.5000,-7374.00,-92.154150
14988,"Lee, Yuan Siong",04:46:36,None,None,21221,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,17196.0,17196.00,-9454.2000,-9340.35000,-9226.5000,-9606.00,-121.561265
14989,"Chia, Peter",05:52:39,None,None,28776,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,21159.0,21159.00,-13417.2000,-13303.35000,-13189.5000,-13569.00,-173.774704
14990,"Wong, Kin Leong",04:36:35,None,None,19528,None,Marathon,None,None,None,...,7741.8000,7855.65000,7969.5000,16595.0,16595.00,-8853.2000,-8739.35000,-8625.5000,-9005.00,-113.642951


In [836]:
# Choose the best performing event for each athlete

#top_performers_clean = excluded_list.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [837]:
# Choose the best result for each event participated by every athlete

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [838]:
top_performers_clean.reset_index(inplace=True)


In [839]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/World Athletics Champs/')


top_performers_clean.to_csv('WAC_top_performers_prod.csv', encoding='utf-8')

In [840]:
top_performers_clean

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,14549,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,...,16.8756,16.61730,16.3590,10.09,10.09,-6.7856,-6.52730,-6.2690,-7.13,-36.405343
1,14592,Zhou Xuanyu,9.12,DHS,nan,10,C,Triple Jump,None,None,...,14.2590,14.04075,13.8225,9.12,9.12,-5.1390,-4.92075,-4.7025,-5.43,-32.319588
2,14700,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,16.8756,16.61730,16.3590,10.47,10.47,-6.4056,-6.14730,-5.8890,-6.75,-34.198606
3,14919,Zheng Justin De,10.29,NJC,None,12,C,Triple Jump,None,None,...,16.8756,16.61730,16.3590,10.29,10.29,-6.5856,-6.32730,-6.0690,-6.93,-35.243902
4,14855,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,16.8756,16.61730,16.3590,11.67,11.67,-5.2056,-4.94730,-4.6890,-5.55,-27.229965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10258,4698,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
10259,5076,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
10260,11445,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
10261,11180,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [841]:
top_performers_clean[top_performers_clean['NAME']=='PEREIRA, VERONICA SHANTI']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
4809,14950,"PEREIRA, VERONICA SHANTI",22.81,None,None,2,None,200m,None,None,...,23.0214,23.35995,23.6985,22.81,22.81,0.2114,0.54995,0.8885,-0.24,3.936642
6732,8122,"PEREIRA, VERONICA SHANTI",11.45,Singapore,29,1,Open,Dash,100,None,...,11.2914,11.45745,11.6235,11.45,11.45,-0.1586,0.00745,0.1735,-0.38,1.567299


In [842]:
'''
# Join 2024 best results for each event for each athlete

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/2023/')


df_yoy = pd.read_csv("Best_results_2023.csv")

df_yoy['NAME'] = df_yoy['NAME'].str.replace('\xa0', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\r', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\n', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.strip()
'''

'\n# Join 2024 best results for each event for each athlete\n\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/2023/\')\n\n\ndf_yoy = pd.read_csv("Best_results_2023.csv")\n\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\xa0\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'[\x00-\x1f\x7f-\x9f]\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\r\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\n\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.strip()\n'

In [843]:
#df_yoy

In [844]:
'''

# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

yoy_performance = pd.merge(
    left=df_yoy, 
    right=top_performers_clean,
    how='left',
    left_on=['EVENT', 'GENDER', 'NAME'],
    right_on=['MAPPED_EVENT', 'GENDER', 'NAME'],
)

'''

"\n\n# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER\n\nyoy_performance = pd.merge(\n    left=df_yoy, \n    right=top_performers_clean,\n    how='left',\n    left_on=['EVENT', 'GENDER', 'NAME'],\n    right_on=['MAPPED_EVENT', 'GENDER', 'NAME'],\n)\n\n"

In [845]:
#yoy_performance

In [846]:
'''

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


yoy_performance.to_csv('yoy_performance_prod.csv', index=False, encoding='utf-8')

'''

"\n\nos.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')\n\n\nyoy_performance.to_csv('yoy_performance_prod.csv', index=False, encoding='utf-8')\n\n"

In [847]:
# Choose best performance for each event

#tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

tiered_performers = top_performers_clean


In [848]:
tiered_performers

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,14549,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,...,16.8756,16.61730,16.3590,10.09,10.09,-6.7856,-6.52730,-6.2690,-7.13,-36.405343
1,14592,Zhou Xuanyu,9.12,DHS,nan,10,C,Triple Jump,None,None,...,14.2590,14.04075,13.8225,9.12,9.12,-5.1390,-4.92075,-4.7025,-5.43,-32.319588
2,14700,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,16.8756,16.61730,16.3590,10.47,10.47,-6.4056,-6.14730,-5.8890,-6.75,-34.198606
3,14919,Zheng Justin De,10.29,NJC,None,12,C,Triple Jump,None,None,...,16.8756,16.61730,16.3590,10.29,10.29,-6.5856,-6.32730,-6.0690,-6.93,-35.243902
4,14855,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,16.8756,16.61730,16.3590,11.67,11.67,-5.2056,-4.94730,-4.6890,-5.55,-27.229965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10258,4698,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
10259,5076,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
10260,11445,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
10261,11180,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [849]:
tiered_performers[tiered_performers['NAME']=='PEREIRA, VERONICA SHANTI']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
4809,14950,"PEREIRA, VERONICA SHANTI",22.81,None,None,2,None,200m,None,None,...,23.0214,23.35995,23.6985,22.81,22.81,0.2114,0.54995,0.8885,-0.24,3.936642
6732,8122,"PEREIRA, VERONICA SHANTI",11.45,Singapore,29,1,Open,Dash,100,None,...,11.2914,11.45745,11.6235,11.45,11.45,-0.1586,0.00745,0.1735,-0.38,1.567299


In [850]:
# Identify Tier 1/2/3 performers

#top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
#                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
#                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))


tiered_performers['TIER'] = np.where((tiered_performers['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((tiered_performers['Delta_Benchmark']<0) & (tiered_performers['Delta2']>=0)), 'Tier 2',
                                np.where(((tiered_performers['Delta2']<0) & (tiered_performers['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((tiered_performers['Delta3.5']<0) & (tiered_performers['Delta5']>=0)), 'Tier 4', ' '))))



In [851]:
tiered_performers

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
0,14549,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,...,16.61730,16.3590,10.09,10.09,-6.7856,-6.52730,-6.2690,-7.13,-36.405343,
1,14592,Zhou Xuanyu,9.12,DHS,nan,10,C,Triple Jump,None,None,...,14.04075,13.8225,9.12,9.12,-5.1390,-4.92075,-4.7025,-5.43,-32.319588,
2,14700,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,16.61730,16.3590,10.47,10.47,-6.4056,-6.14730,-5.8890,-6.75,-34.198606,
3,14919,Zheng Justin De,10.29,NJC,None,12,C,Triple Jump,None,None,...,16.61730,16.3590,10.29,10.29,-6.5856,-6.32730,-6.0690,-6.93,-35.243902,
4,14855,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,16.61730,16.3590,11.67,11.67,-5.2056,-4.94730,-4.6890,-5.55,-27.229965,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10258,4698,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
10259,5076,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
10260,11445,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
10261,11180,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,


In [852]:
tiered_performers[tiered_performers['NAME']=='PEREIRA, VERONICA SHANTI']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
4809,14950,"PEREIRA, VERONICA SHANTI",22.81,None,None,2,None,200m,None,None,...,23.35995,23.6985,22.81,22.81,0.2114,0.54995,0.8885,-0.24,3.936642,Tier 2
6732,8122,"PEREIRA, VERONICA SHANTI",11.45,Singapore,29,1,Open,Dash,100,None,...,11.45745,11.6235,11.45,11.45,-0.1586,0.00745,0.1735,-0.38,1.567299,Tier 3


In [853]:
# Drop rows without a benchmark

final_df = tiered_performers[tiered_performers['BENCHMARK'].notna()]


In [854]:
final_df[final_df['NAME']=='PEREIRA, VERONICA SHANTI']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
4809,14950,"PEREIRA, VERONICA SHANTI",22.81,None,None,2,None,200m,None,None,...,23.35995,23.6985,22.81,22.81,0.2114,0.54995,0.8885,-0.24,3.936642,Tier 2
6732,8122,"PEREIRA, VERONICA SHANTI",11.45,Singapore,29,1,Open,Dash,100,None,...,11.45745,11.6235,11.45,11.45,-0.1586,0.00745,0.1735,-0.38,1.567299,Tier 3


In [855]:
# Process dates to extract age

# Map NSG divisions into age

mask = (final_df['DIVISION'].str.contains(r'A', na=False))
final_df.loc[mask, 'AGE'] = '18.5'

mask = (final_df['DIVISION'].str.contains(r'B', na=False))
final_df.loc[mask, 'AGE'] = '16'

mask = (final_df['DIVISION'].str.contains(r'C', na=False))
final_df.loc[mask, 'AGE'] = '13.5'

mask = (final_df['DIVISION'].str.contains(r'O', na=False))
final_df.loc[mask, 'AGE'] = '12'



In [856]:
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


final_df['DOB_new'] = final_df['DOB'].apply(length)



#B = parser.parse("10-09-2021", dayfirst = True)

In [857]:
final_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')

final_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')

final_df['year_extract'] = pd.to_numeric(final_df['year_extract'])

final_df['age_extract'] = 2025 - final_df['year_extract']


In [858]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


final_df['age_extract']=final_df['age_extract'].apply(age)


In [859]:
# If NSG event then choose AGE otherwise choose age_extract

condition1 = final_df['COMPETITION']=='National School Games'
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)


In [860]:
# Change to numeric

final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)

In [861]:
final_df[final_df['NAME']=='PEREIRA, VERONICA SHANTI']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
4809,14950,"PEREIRA, VERONICA SHANTI",22.81,None,None,2,None,200m,None,None,...,22.81,0.2114,0.54995,0.8885,-0.24,3.936642,Tier 2,1996-09-20,1996.0,29.0
6732,8122,"PEREIRA, VERONICA SHANTI",11.45,Singapore,12,1,Open,Dash,100,None,...,11.45,-0.1586,0.00745,0.1735,-0.38,1.567299,Tier 3,1996-09-20,1996.0,29.0


In [862]:
final_df = final_df[(final_df['TIER']!=' ') & (final_df['TIER'] is not None)]


In [863]:
final_df[final_df['NAME']=='PEREIRA, VERONICA SHANTI']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
4809,14950,"PEREIRA, VERONICA SHANTI",22.81,None,None,2,None,200m,None,None,...,22.81,0.2114,0.54995,0.8885,-0.24,3.936642,Tier 2,1996-09-20,1996.0,29.0
6732,8122,"PEREIRA, VERONICA SHANTI",11.45,Singapore,12,1,Open,Dash,100,None,...,11.45,-0.1586,0.00745,0.1735,-0.38,1.567299,Tier 3,1996-09-20,1996.0,29.0


In [865]:
final_df

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
412,11208,"LOW, JUN YU",5.20m,NUS,12,1,Open,Pole Vault,0,,...,5.20,-0.0332,0.04690,0.1270,-0.14,2.378277,Tier 3,2001-04-21,2001.0,24.0
1635,8618,Kampton Kam,2.25,None,None,1,None,High Jump,None,None,...,2.25,-0.0334,0.00155,0.0365,-0.08,1.566524,Tier 3,2001-03-06,2001.0,24.0
2823,3235,"QUEK, JUN JIE CALVIN",50.77,None,None,1,None,400m Hurdles,None,None,...,50.77,-1.3000,-0.57250,0.1550,-2.27,0.319588,Tier 4,1996-02-26,1996.0,29.0
4809,14950,"PEREIRA, VERONICA SHANTI",22.81,None,None,2,None,200m,None,None,...,22.81,0.2114,0.54995,0.8885,-0.24,3.936642,Tier 2,1996-09-20,1996.0,29.0
6274,9465,"ANG, CHEN XIANG",13.8,Singapore,12,1,Open,Hurdles,110,None,...,13.80,-0.2646,-0.06555,0.1335,-0.53,1.006029,Tier 4,1994-07-03,1994.0,31.0
6563,314,TATE TAN FUNG,10.5,None,None,1,None,100m,None,None,...,10.50,-0.3000,-0.15000,0.0000,-0.50,0.000000,Tier 4,2005-05-17,2005.0,20.0
6732,8122,"PEREIRA, VERONICA SHANTI",11.45,Singapore,12,1,Open,Dash,100,None,...,11.45,-0.1586,0.00745,0.1735,-0.38,1.567299,Tier 3,1996-09-20,1996.0,29.0
6908,74,"Louis, Marc Brian",10.43,None,None,3,None,100m,None,None,...,10.43,-0.2300,-0.08000,0.0700,-0.43,0.700000,Tier 4,2002-08-07,2002.0,23.0
7686,57,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,...,1871.40,24.9840,52.87200,80.7600,-12.20,4.343804,Tier 2,1991-05-17,1991.0,34.0


In [864]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/World Athletics Champs/')


final_df.to_csv('WAC_tiered_performers.csv', encoding='utf-8')